In this notebook, I implemented algorithms 
- with dynamic programming to calculate edit distance for approximate matching between a pattern and a text; 
- to find maximum overlaps between sequence reads, and 
- to assemble a virus genome (greedy shortest common string algorithm

### 1. Calculating minimum edit distance between a pattern p and a text t

Dynamic programming is employed here, to decompose a recursive problem into smaller problems, while avoiding redundantly recalculating one of these smaller problems.  
A matrix D of dimension p*t is initialized, with first row filled with zeros and first column filled with range(len(p)).  
The edit distance for each cell is calculative progressively. The minimum edit distance of p and t will be the minimum value at the last row of D. 

In [13]:
def edit_distance(p, t):
    '''Function to calculate minimum edit distance between a pattern p and a text t'''
    
    # Initialize instance matrix and fill with zeros
    D = []
    for i in range(len(p)+1): 
        D.append([0]*(len(t)+1))
    for i in range(len(p)+1):    # Initialize first column. First row will be left as zeros
        D[i][0] = i
    
    # Fill in rest of the matrix:
    for i in range(1, len(p)+1):
        for j in range(1, len(t)+1):
            edist_horizontal = D[i][j-1] +1 
            edist_vertical = D[i-1][j] + 1
            
            if p[i-1] == t[j-1]:
                 edist_diagonal = D[i-1][j-1]
            else:
                 edist_diagonal = D[i-1][j-1] + 1
            D[i][j] = min(edist_horizontal, edist_vertical, edist_diagonal)
    return min(D[-1])

In [14]:
edit_distance('avd','abcde')

2

In [15]:
p = 'GCGTATGC'
t = 'TATTGGCTATACGGTT'

In [16]:
edit_distance(p,t)

2

##### Test case with human genome ch1 

In [27]:
from functions.parse_genome import parse_fasta, parse_fastq
# parse human genome data chr1
human = parse_fasta('data/chr1.GRCh38.excerpt.fasta')

In [22]:
p = 'GCTGATCGATCGTACG'
t = human
edit_distance(p,t)

3

In [23]:
p = 'GATTTACCAGATTGAG'
t = human
edit_distance(p,t)

2

### 2. Algorithm to find the longest exact overlap (suffix/prefix match) between two strings

In [94]:
# Helper function
def overlap(a, b , k):
    """Returns length of longest suffix of 'a' in prefix of 'b' that is at least min_length
    If no such overlaps returns 0"""
    start = 0
    while True:
        start = a.find(b[:k], start)
        if start == -1:
            return 0
        if b.startswith(a[start:]):
            return len(a) - start
        start += 1 

In [197]:
# a naive algorithm which compares permutation of every substring and find overlaps
def naive_overlap_map(reads, k):
    """Find overlaps among every pair of reads"""
    overlaps = {} 
    for a, b in permutations(reads,2):
        overlap_length = overlap(a, b, min_length=k)
        if overlap_length > 0:
            overlaps[(a, b)] = overlap_length
    return overlaps

The naive_overlap_map function becomes very slow as number of reads increases  
Here I optimized this algorithm by removing duplicated comparisons of a and b.  

- The idea is to firstly create a dictionary, with keys as a k-mer of each read, values (set) as the id of reads which contain the same k-mer.
- Next, for each read, we query the dictionary with its suffix kmer, which returns reads that contain the same kmer. We compare the read with these sequences only (but remove the read itself).


In [103]:
# Helper function to build a dictionary of kmers
def read_kmers(reads, k):
    '''Find all distinct kmers for each sequence read of the parsed fastq file
    reads - list of sequences
    k - length of kmers'''
    from collections import defaultdict
    kmers = defaultdict(set)
    for id, read in enumerate(reads):
        for i in range(len(read)-k + 1):
            kmers[read[i:i+k]].add(id)           
    return kmers
        

In [140]:
def overlap_map(reads, k):
    """Find overlaps among every pair of reads
    only 'b' with kmers exist in 'a' will be searched
    Thus will be much more efficient than the naive_overlap_map function """
    overlaps = {}
    kmers = read_kmers(reads, k)
    for a in range(len(reads)):
        reads_to_compare = list(kmers[reads[a][-k:]])
        for b in reads_to_compare:
            if a != b:
                overlap_length = overlap(reads[a], reads[b], k)
                if overlap_length > 0:
                    overlaps[(a,b)] = overlap_length
    return overlaps
        

In [126]:
# Test case 1: 10000 reads of a virus genome
phix =  parse_fastq('data/ERR266411_1.for_asm.fastq')[0] # extract only sequence data
phix_overlaps = overlap_map(phix, k=30)

In [132]:
len(phix_overlaps) # number of edges for the graph

904746

In [139]:
len(set([ a for a, b in phix_overlaps.keys()])) # number of nodes for the graph

7161

In [142]:
# Test case 2
reads = ['ABCDEFG', 'EFGHIJ', 'HIJABC']
overlap_map(reads, k=3)

{(0, 1): 3, (1, 2): 3, (2, 0): 3}

### 3. Assembly problem: how to find the shorest common substring among all reads?


#### A naive algorithm which trys permutation of all substrings to find the shortest common substrings among all reads
Note: the naive_overlap_map algorithm is for finding longest overlaps between every pair of strings (of all reads)

In [152]:
def scs(ss):
    from itertools import permutations
    from collections import defaultdict
    '''find shortest common super strings, keep track of different resulted strings'''
    shortest_s = None
    sup_dict = defaultdict(list)
    for perm in permutations(ss):
        sup = perm[0]
        
        for i in range(len(perm)-1):
            
            overlap_len = overlap(perm[i],perm[i+1], k=1)
            sup += perm[i+1][overlap_len:]
        sup_dict[len(sup)].append(shortest_s)
        
        if shortest_s == None or len(shortest_s) > len(sup):
            shortest_s = sup

    return shortest_s, sup_dict


In [153]:
# Test case 1 
ss = ['CCT','CTT','TGC','TGG','GAT','ATT']
shortest_s, sup_dict = scs(ss)

In [156]:
# Finding variations of the shortest common substrings
sup_dict[len(shortest_s)]

['CCTTGCTGGATT', 'CCTTGGATTGC', 'CCTTGGATTGC', 'CCTTGGATTGC']

### Assemble an unkown virus genome, using an optimized greedy scs algorithm
Again, the naive algorithm which uses permutation methods is very slow as the number of reads increases.  
We shall use a greedy algorithm combined with the kmer-dictionary (as in section 2) approach to speed up.

In [191]:
# 1) Define a helper function which returns maximum suffix/prefix overlap between two reads

def max_overlap(reads, k):
    '''Function which returns maximum suffix/prefix > length k overlap between two reads.
      Returns 0 if no overlap'''
    reada, readb = None, None
    max_overlap = 0
    overlaps = {}
    kmers = read_kmers(reads, k)
    
    for a in range(len(reads)):
        reads_to_compare = list(kmers[reads[a][-k:]])
        for b in reads_to_compare:
            if a != b:
                overlap_len = overlap(reads[a], reads[b], k)
        
                if overlap_len > max_overlap:
                    reada, readb, max_overlap = reads[a], reads[b], overlap_len
    
    return reada, readb, max_overlap


In [192]:
max_overlap(virus, k=30)

('GCTCATGAAATCCTGGATCATAGTGTCACAGGGGCAAGAGAGTCTATTGCAGGCATGCTAGATACCACAAAAGGCCTGATTCGAGCCAGCATGAGGAAGG',
 'CTCATGAAATCCTGGATCATAGTGTCACAGGGGCAAGAGAGTCTATTGCAGGCATGCTAGATACCACAAAAGGCCTGATTCGAGCCAGCATGAGGAAGGG',
 99)

In [193]:
# 2) Implement the Greedy common shortest string algorithm.

def greedy_scs(reads, k):
    '''Function to find the common shortest string from all reads
    It loops and concatenates every pair of reads which have the maximum overlap'''
    reada, readb, overlap = max_overlap(reads, k) # Note this line appeared twice, here is to initiate values
    while overlap > 0:
        reads.remove(reada)
        reads.remove(readb)
        reads.append(reada + readb[overlap:])
        reada, readb, overlap = max_overlap(reads, k)
    return ''.join(reads) # concatenate the rest reads, if no overlaps
    

In [198]:
# Test case 1, use a mystery virus genome
virus =  parse_fastq('data/ads1_week4_reads.fq')[0] # extract only sequence data

In [162]:
virus[0]

'GTCCAGCAGAGCAAGTGATGCGAGAGCTGCCCATCCTCCAACCAGCATGCCCCTAGACATTGACACTGCATCGGAGTCAGGCCAAGATCCGCAGGACAGT'

In [163]:
len(virus)

1881

#### Finally, the genome assembly

In [194]:
# Run algorithm to assemble virus genome and time it
start = time.time()
genome = greedy_scs(virus, k=30)
end = time.time()
elapsed = end - start
print(elapsed) 
# Took about 90 seconds to run

90.27461886405945


In [195]:
# Length of assembled genome
len(genome)

15894

In [196]:
# Inspect results
from collections import Counter
Counter(genome)

Counter({'A': 4633, 'C': 3789, 'G': 3749, 'T': 3723})